In [8]:
import os
import logging 
import re


###任务描述
###分步骤描述
###工具插件描述
/*通过清晰地分步骤描述任务，并提供必要的工具插件信息，有效利用LLM完成具体任务，生成每一步的代码。*/
/*确保输入结构清晰，详细描述每个步骤的要求和工具的使用方法。使得模型的输出将更加准确和实用。*/

whole-> 3个stage 实现
FOR EACH STAGE:
  1/输入初步任务描述及可使用工具信息
      - llm返回任务基本描述及规划步骤
      - 人工修改步骤并确认
  2/每一步骤逐步向llm输入并获取response代码 
      - 代码块增量进行测试
      - 代码在10个用例上测试通过进行步骤代码留存
      - 不通过，返回报错 修改代码
  3/迭代完单stage 进行批量数据测试

In [9]:
import logging
def clear_existing_loggers():
    """Clear all existing loggers to prevent conflicts."""
    for handler in logging.root.handlers[:]:
        logging.root.removeHandler(handler)

def setup_logger(logfile_path):
    """Setup logger to always print time and level."""
    clear_existing_loggers()
    logging.basicConfig(
        format='%(asctime)s %(levelname)-8s %(message)s',
        level=logging.INFO,
        datefmt='%Y-%m-%d %H:%M:%S',
        handlers=[
            logging.FileHandler(logfile_path),
            logging.StreamHandler()
        ]
    )

def log_w_indent(text, indent, flag, symbol='>>'):
    """Log and add indent."""
    ind2col = {i: f"\x1b[{a}m" for i, a in enumerate([
        '1', '31', '33', '34', '35'])}
    reset = "\x1b[0m"

    if indent < 0 or indent >= len(ind2col):
        raise ValueError("Invalid indent level")

    if indent > 0:
        if flag == 'Question':
            prefix = '  Q:'
        elif flag == 'Response':
            prefix = '  LLM-A:'
        elif flag == 'Code':
            prefix = '  Code script test:'
        else:
            raise ValueError("Invalid flag")

        prefix += ind2col[indent] + (indent * 2) * symbol + ' ' + text + reset
    else:
        prefix = ind2col[indent] + text + reset

    logging.info(prefix)

setup_logger(r"D:\GXQ\human_seg\neuron_seg_human\llm_neuron_seg\utils\log_utils\log0720.log")
log_w_indent("This is a test question", 1, "Question")
log_w_indent("This is a test response", 1, "Response")

2024-07-22 10:38:26 INFO       Q:>>>> This is a test question
2024-07-22 10:38:26 INFO       LLM-A:>>>> This is a test response


In [10]:
#配置openai客户端
from openai import OpenAI
# from openai.error import APIConnectionError
#from dotenv import load_dotenv
import openai
import os
import json
#config.json 进行配置用户openai账户 
with open(r'D:\GXQ\human_seg\neuron_seg_human\llm_neuron_seg\utils\llm\llm_config.json') as config_file:
    config = json.load(config_file)

CLIENT = OpenAI(base_url=config['OPENAI_API_BASE'],
                api_key=config['OPENAI_API_KEY'])

In [11]:
# 分离文本+提取代码块
response = """
Here is an example of Python code:

```python
import pandas as pd

# Read CSV file
data = pd.read_csv('input.csv')
print(data.head())```
You can use the above code to read a CSV file.

Another code snippet:

python
复制代码
from data_cleaning_tool import clean_data

# Clean data
cleaned_data = clean_data(data)
print(cleaned_data.head())
Make sure to install the necessary libraries before running the code.
"""


def extract_code_blocks(response):
    # 正则表达式匹配代码块
    code_pattern = re.compile(r'```(?:\w+)?\n(.*?)```', re.DOTALL)
    code_blocks = code_pattern.findall(response)
    non_code_content = code_pattern.sub('', response)
    non_code_content = non_code_content.strip()
    
    return code_blocks, non_code_content

# 使用函数提取代码块和其他内容
code_blocks, non_code_content = extract_code_blocks(response)

print("Code Blocks:")
for i, code in enumerate(code_blocks, 1):
    print(f"Code Block {i}:\n{code}\n")

print("Non-Code Content:")
print(non_code_content)


Code Blocks:
Code Block 1:
import pandas as pd

# Read CSV file
data = pd.read_csv('input.csv')
print(data.head())

Non-Code Content:
Here is an example of Python code:


You can use the above code to read a CSV file.

Another code snippet:

python
复制代码
from data_cleaning_tool import clean_data

# Clean data
cleaned_data = clean_data(data)
print(cleaned_data.head())
Make sure to install the necessary libraries before running the code.


In [27]:
import hashlib
from collections import defaultdict
import subprocess
import json
import pickle

def read_json_file(file_path):
    # FoundError: 如果文件不存在。
    # ValueError: 如果文件为空或 JSON 解析失败。
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"The file {file_path} does not exist.")

    with open(file_path, 'r') as file:
        data = file.read()
        if not data:
            raise ValueError("The JSON file is empty.")
        try:
            json_data = json.loads(data)
            print(f"Parsed JSON content from {file_path}:")
            return json_data
        except json.JSONDecodeError as e:
            raise ValueError(f"Failed to decode JSON from {file_path}: {e}")

template_file_path = r'D:\GXQ\human_seg\neuron_seg_human\llm_neuron_seg\utils\llm\instruct_template\template.json'
code_tools_file_path = r'D:\GXQ\human_seg\neuron_seg_human\llm_neuron_seg\utils\llm\code_tools.json'
template_json = read_json_file(template_file_path)
codetools_json = read_json_file(code_tools_file_path)

class SpoofData:
    def __getitem__(self, item):
        return f'<{item}>'
def md5hash(string):
    return int(hashlib.md5(string.encode('utf-8')).hexdigest(), 16)
#llm问答
class LLMResponse:
    def __init__(self,ini_task:str,code_save_dir):
        self.ini_task_question=ini_task
        self.ini_step_description=None
        self.step_list = None
        self.manual_list=None
        # self.n_regenerate=None
        # self.reset=reset
        #      #迭代十次不通过 记录step_id 在reset 
        # self.reset_stages=reset_stages
        #self.export_predict_response = defaultdict(defaultdict(list).copy)
        self.code_script=None
        self.code_save_dir = code_save_dir
        
    def save_state(self, filename):
        with open(filename, 'wb') as f:
            pickle.dump(self, f)

    @staticmethod
    def load_state(filename):
        with open(filename, 'rb') as f:
            return pickle.load(f)   
        
    #indent用于日志记录 stepid:步骤id  reuse:是否进行再次输入迭代
    def llw_w_log(self, prompt, indent,stepidx, stage,reuse=False):
        #reset.stages 重置阶段列表     拿到需要重新run的stepidx
        if (stage in self.reset_stages) and (reset := self.reset.get(stepidx, {})):
            if self.accept_restore_failure and not len(reset[md5hash(prompt)]):
                logging.warning('Spoofing aborted!')
                predict_response =llm_w_log(prompt, indent)
            else:
                log_w_indent(f'Spoofed Input: {prompt}', indent,flag='Question')
                predict_response= reset[md5hash(prompt)].pop(0)
                log_w_indent(f'Spoofed Output: {predict_response}', indent, flag='Question',symbol='xx')
    
        else:
            predict_response = llm_w_log(prompt, indent)

        self.export_predict_response[stepidx][md5hash(prompt)].append(predict_response)

        return predict_response
    # 生成step列表 eachstep 进行prompt修饰后 进行llm输入，获取处理代码

    def get_ini_task_decription(self) -> str:
        log_w_indent(f'Input_Question: {self.ini_task_question}', indent=0 , symbol='<<<<<<',flag='Question')
        self.ini_task_description = chat_llm(prompt='Task:'+self.ini_task_question+template_json['get_ini_task'])
        log_w_indent(f'Initial_task_description: {self.ini_task_description}', indent=1 , flag='Question')
        self.save_state("state_after_get_ini_task_description.pkl")
        
    def extract_steplist(self, text):
        pattern = re.compile(r'(\d+)\.\s*(.*?)(?=(\d+\.|$))', re.DOTALL)
        matches = pattern.findall(text)
        paragraph_dict = {f's{num}': content.strip() for num, content, _ in matches}
        return paragraph_dict
    
    
    def get_task_step(self) -> str:
        prompt = 'Task:' + self.ini_task_question + template_json['get_step_list']
        log_w_indent(f'Task_step_decom_prompt: {prompt}', indent=0 , flag='Question')
        step_list = chat_llm(prompt)
        
        log_w_indent(f'Initial_task_step: {step_list}', indent=1 , flag='Response')
        step_clean_list = self.extract_steplist(step_list)
        self.step_list=step_clean_list
        log_w_indent(f'Initial_clean_task_step: {step_clean_list}', indent=1 , flag='Response')
        self.save_state("state_after_get_task_step.pkl")
    
    # manual 
    def manual_modify_step(self) -> list:
        ### human modify step_list ###
        step_prompt_list = self.step_list
        step_modify_list = []
        print("Original step_prompt_list:")
        for key, step in step_prompt_list.items():
            print(f"{key}. {step}")
        print("\nPlease enter the modified steps. Press Enter to keep the original step.\n")

        # 循环获取用户输入
        for key, step in step_prompt_list.items():
            modified_step = input(f"Modify the step '{step}': ")
            if modified_step.strip():  # 如果输入不为空，去除前后的空格
                step_modify_list.append((key, modified_step))
            else:  
                step_modify_list.append((key, step))
        
        self.manual_list = {key: step for key, step in step_modify_list}
        log_w_indent(f'Manual_step_list: {self.manual_list}', indent=2 , flag='Response')
        self.save_state("state_after_manually_modify_step.pkl")
    

       
    def extract_code_blocks(self,self_response):
    # 正则表达式匹配代码块
        code_pattern = re.compile(r'```(?:\w+)?\n(.*?)```', re.DOTALL)
        code_blocks = code_pattern.findall(response)
        non_code_content = code_pattern.sub('', response)
        non_code_content = non_code_content.strip()
        self.code_script = code_blocks
       
    
        return code_blocks, non_code_content
    
    def get_step_code(self, step_id):
    # stepid 以 s1/s2...
        task = self.ini_step_description
        step_list = self.manual_list
        pre_code = self.code_script if self.code_script is not None else ""
        current_step = step_list[step_id] if step_id < len(step_list) else None
    
        if current_step is None:
            raise ValueError(f"Step ID {step_id} is out of range in the step list.")
    
        code_tools = codetools_json
    # step_code 要改 json无法
        prompt = (template_json['get_step_code'] + 
              f'{task}\nHere is the step-by-step plan: {step_list}\n'
              f'Here is the script of pre_steps: {pre_code}\n'
              f'Here is the current step: {current_step}\n'
              f'Please prioritize using the following code tools and libraries if they fit the step requirements: {code_tools}\n'
              'Provide only the whole code script and return the script only.')
    
        log_w_indent(f'{step_id}_code_question: {prompt}', indent=0, flag='Question')
    
        try:
            step_response = chat_llm(prompt)
            step_code, _ = extract_code_blocks(self, step_response)
            self.code_script = step_code
            log_w_indent(f'{step_id}_code_script: {self.code_script}', indent=2, flag='Code')
            return step_response
        except Exception as e:
            log_w_indent(f'Error in getting step code: {str(e)}', indent=2, flag='Error')
            raise



    # def get_all_prompts_for_log(self):
    #     # Spoof data to log prompting format.
    #     data = SpoofData()
    #     prompts = dict(
    #         gen_ini_task_decription=self.gen_ini_task_decription(data),
    #         task_step_list= self.gen_task_step(data),
    #         manual_modify_list = self.manual_modify_step(data),
    #         step_description=  self.extract_code_blocks(data),
    #         # 步骤额外信息也需记录
    #         # step_response =self.get_step_code(step_response)),
    #         # step_code,step_code_description =extract_code_blocks(data)
    #         )
    #     #logger记录所有
    
    #     return prompts

    def run_script(self, input_data, step_id, output_suffix):
        code = self.code_script
        script_dir = self.code_save_dir
        file_path = os.path.join(script_dir, f'{step_id}_script.py')
        output_file_path = os.path.join(script_dir, f'{step_id}_output.{output_suffix}')
        
        try:
            with open(file_path, "w") as file:
                file.write(code)
        except Exception as e:
            return f"Error writing to file: {e}"

        cmd_command = f"python {file_path} {input_data}"
        try:
            result = subprocess.run(cmd_command, shell=True, capture_output=True, text=True)
            result.check_returncode()  # 检查命令是否成功运行
            logging.info(f"Code script generated successfully!!{file_path}:{result.stdout}")
            return result.stdout
        except subprocess.CalledProcessError as e:
            logging.info(f"Error running script: {e.stderr.strip()}")
            return e.stderr.strip()
        except Exception as e:
            logging.info(f"Unexpected error: {str(e)}")
            return str(e)
        
    # def pass_metric(self):



ques = 'How to generate training masks for tracing neuron structures from 3D TIF images of human brain neurons??'
code_save_dir = r"D:\GXQ\human_seg\neuron_seg_human\llm_neuron_seg\utils\llm"
# llm_test = LLMResponse(ini_task=ques,code_save_dir=code_save_dir)
# llm_test.get_ini_task_decription()
# llm_test.get_task_step()
llm_test = LLMResponse.load_state("state_after_get_task_step.pkl")
llm_test.manual_modify_step()
llm_test.get_step_code(step_id='s1')



 


Parsed JSON content from D:\GXQ\human_seg\neuron_seg_human\llm_neuron_seg\utils\llm\instruct_template\template.json:
Parsed JSON content from D:\GXQ\human_seg\neuron_seg_human\llm_neuron_seg\utils\llm\code_tools.json:
Original step_prompt_list:
s1. Load the 3D TIF image using an appropriate library.
s2. Preprocess the image to enhance its quality for segmentation.
s3. Apply a segmentation algorithm to identify neuron structures.
s4. Post-process the segmented image to refine the results.
s5. Generate binary masks from the segmented image.
s6. Save the binary masks as TIF files for training purposes.

Key points the final script should address include:
- Efficiently handling 3D image data.
- Applying suitable preprocessing and segmentation techniques.
- Refining the segmentation results to improve mask quality.
- Ensuring the binary masks are correctly saved for training use.

Please enter the modified steps. Press Enter to keep the original step.



2024-07-22 11:07:30 INFO       LLM-A:>>>>>>>> Manual_step_list: {'s1': 'Load the 3D TIF image using an appropriate library.', 's2': 'Preprocess the image to enhance its quality for segmentation.', 's3': 'Apply a segmentation algorithm to identify neuron structures.', 's4': 'Post-process the segmented image to refine the results.', 's5': 'Generate binary masks from the segmented image.', 's6': 'Save the binary masks as TIF files for training purposes.\n\nKey points the final script should address include:\n- Efficiently handling 3D image data.\n- Applying suitable preprocessing and segmentation techniques.\n- Refining the segmentation results to improve mask quality.\n- Ensuring the binary masks are correctly saved for training use.'}


AttributeError: 'LLMResponse' object has no attribute 'code_script'

In [ ]:
import re

def extract_paragraphs(text):
    # 使用正则表达式匹配以数字加句点开头的段落
    pattern = re.compile(r'(\d+)\.\s*(.*?)(?=(\d+\.|$))', re.DOTALL)
    matches = pattern.findall(text)
    
    # 创建字典，数字作为key，段落内容作为value
    paragraph_dict = {f's{num}': content.strip() for num, content, _ in matches}
    
    return paragraph_dict

# 示例文本
text = """
1. **Install Required Libraries**
   - **Description**: Set up the Python environment and install the necessary libraries. Common libraries for handling 3D image data and processing neurons include `numpy`, `scipy`, `skimage`, `tifffile`, `neuroglancer`, and `matplotlib`.
   - **Tools**: `pip` for installation.

2. **Load 3D TIF Image**
   - **Description**: Load the 3D TIF image into memory for processing. Utilize libraries like `tifffile` to handle the image file.
   - **Tools**: `tifffile` for reading the TIF file.

3. **Preprocess Image**
   - **Description**: Preprocess the image to enhance features of the neurons. This could involve normalization, denoising, and contrast enhancement.
   - **Tools**: `skimage` for preprocessing functions such as `filters` and `exposure`.

4. **Segment Neurons**
   - **Description**: Apply segmentation algorithms to distinguish neurons from the background. Common methods include thresholding, edge detection, and morphological operations.
   - **Tools**: `skimage` for segmentation tasks with algorithms like `threshold_otsu`, `canny`, and `morphology`.

5. **Skeletonize Neurons**
   - **Description**: Reduce the segmented neurons to a skeletal representation to simplify the structure while retaining meaningful connectivity.
   - **Tools**: `skimage.morphology.skeletonize_3d`.

6. **Trace Neuron Paths**
   - **Description**: Develop a tracing algorithm to follow along the skeleton of the neurons, potentially creating a graph representation of the structure.
   - **Tools**: Custom Python functions leveraging `numpy` for array manipulation.

7. **Visualize Neuron Structure**
   - **Description**: Visualize the neuron structures in 3D to confirm accuracy and for further analysis.
   - **Tools**: `matplotlib` (3D plotting), `neuroglancer`, or other 3D visualization tools.

8. **Extract and Save Neuron Data**
   - **Description**: Extract meaningful data from the neuron structures, such as branch points and connection pathways. Save this data in a structured format (e.g., CSV, JSON).
   - **Tools**: `numpy`, `pandas` for data manipulation and saving.
"""

# 提取段落并保存到字典中
paragraphs = extract_paragraphs(text)
print(paragraphs['s7'])
# 打印字典
# for key, value in paragraphs.items():
#     print(f'{key}: {value}\n')


**Visualize Neuron Structure**
   - **Description**: Visualize the neuron structures in 3D to confirm accuracy and for further analysis.
   - **Tools**: `matplotlib` (3D plotting), `neuroglancer`, or other 3D visualization tools.


In [23]:
import openai

# 配置api日志记录
logging.basicConfig(level=logging.WARNING, filename='chat_llm.log', filemode='a',
                    format='%(asctime)s - %(levelname)s - %(message)s')
from tenacity import (retry, wait_random_exponential)
@retry(wait=wait_random_exponential(min=1, max=10))


def chat_llm(prompt,temperature=0.5):
    """Predict with GPT-4 model."""

    if isinstance(prompt, str):
        messages = [
            {"role": "user", "content": prompt},
        ]
    else:
        messages = prompt

    try:
        output = CLIENT.chat.completions.create(
            model='gpt-4o',
            messages=messages,
            temperature =temperature
        )
        response = output.choices[0].message.content
        return response
    except openai.error.APIConnectionError as e:
        # 记录日志警告及错误信息
        logging.warning(f"APIConnectionError occurred: {e}")
        return "An external error occurred while trying to connect to the API. Please try again later."


def main(prompt):
    """ log inputs and outputs."""
    log_w_indent(f'Input: {prompt}', indent=1 , flag='Question')
    response = chat_llm(prompt)
    ##处理response
    log_w_indent(f'Output: {response}', indent=2, symbol='xx',flag='Response')
    return response
# main('how to trace and restruct human neuron structure based on 3d tif images?')